# 문제정의
문제 유형: 회귀 문제 (Regression Problem)
목표: 주어진 제품 및 매장 특성을 기반으로 판매 금액(Item_Outlet_Sales)을 예측한다.
입력 값 (Features): 제품 + 매장 특성
출력 값 (Target): 판매 금액 (연속형 수치)

## 주어진 제품 및 매장 특성들을(X값, 컬럼) 기반으로 판매 금액(Item_Outlet_Sales - 연속형 데이터)를 예측하기

# 1. 라이브러리 및 데이터 불러오기

In [1]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
# train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch4/train.csv")
# test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch4/test.csv")
train
test

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDO38,17.250,Low Fat,0.073251,Canned,76.4986,OUT017,2007,NaN,Tier 2,Supermarket Type1
1,FDO32,6.360,Low Fat,0.120544,Fruits and Vegetables,45.8060,OUT046,1997,Small,Tier 1,Supermarket Type1
2,FDT14,10.695,Regular,0.128247,Dairy,121.8440,OUT018,2009,Medium,Tier 3,Supermarket Type2
3,FDG50,7.405,Low Fat,0.015296,Canned,89.7146,OUT049,1999,Medium,Tier 1,Supermarket Type1
4,FDR11,10.500,Regular,0.142760,Breads,159.3578,OUT049,1999,Medium,Tier 1,Supermarket Type1
...,...,...,...,...,...,...,...,...,...,...,...
1700,NCU18,NaN,Low Fat,0.097769,Household,142.4496,OUT019,1985,Small,Tier 1,Grocery Store
1701,FDY02,NaN,Regular,0.153457,Dairy,264.0910,OUT019,1985,Small,Tier 1,Grocery Store
1702,FDI09,20.750,Regular,0.129864,Seafood,240.1880,OUT018,2009,Medium,Tier 3,Supermarket Type2
1703,DRH23,14.650,Low Fat,0.171282,Hard Drinks,54.0614,OUT017,2007,NaN,Tier 2,Supermarket Type1


# 2.탐색적 데이터 분석(EDA)
구조 확인(ex> 이상치, 결측치)

In [16]:
#----
#샘플링 해서 데이터 확인
#---
# train.head()
# train.tail()
# train.sample(5)

#Item_Identifier, Outlet_Identifier와 같은 식별값들은 필요 없을 듯! drop해서 삭제

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
4356,NCB19,6.525,LF,0.090279,Household,86.7882,OUT035,2004,Small,Tier 2,Supermarket Type1,1975.4286
2368,FDX56,NaN,Regular,0.073701,Fruits and Vegetables,207.1638,OUT027,1985,Medium,Tier 3,Supermarket Type3,7247.2330
6663,NCK54,12.150,Low Fat,0.029569,Household,115.0150,OUT049,1999,Medium,Tier 1,Supermarket Type1,2912.8750
1183,FDG29,17.600,Low Fat,0.056281,Frozen Foods,40.3454,OUT035,2004,Small,Tier 2,Supermarket Type1,796.9626
1558,FDV07,NaN,Low Fat,0.031131,Fruits and Vegetables,111.0228,OUT027,1985,Medium,Tier 3,Supermarket Type3,3205.1612


In [15]:
train.shape, test.shape
#(6818,12) , (1705,11)

# 8:2나 9:1이면 적당함!

((6818, 12), (1705, 11))

In [10]:
train.info() # Item_Outlet_Sales가 y값
             # 타깃값 : 해당 상품이 특정 매장에서 기록한 실제 판매액
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6818 non-null   object 
 1   Item_Weight                5656 non-null   float64
 2   Item_Fat_Content           6818 non-null   object 
 3   Item_Visibility            6818 non-null   float64
 4   Item_Type                  6818 non-null   object 
 5   Item_MRP                   6818 non-null   float64
 6   Outlet_Identifier          6818 non-null   object 
 7   Outlet_Establishment_Year  6818 non-null   int64  
 8   Outlet_Size                4878 non-null   object 
 9   Outlet_Location_Type       6818 non-null   object 
 10  Outlet_Type                6818 non-null   object 
 11  Item_Outlet_Sales          6818 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 639.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIn

In [17]:
# 수치형 데이터 확인
train.describe() # 수치형 데이터의 이상치를 확인하거나 중앙값, 표준편차, 평균 확인
                 # 데이터가 치우쳐져있는 정도 확인 가능

                 # mean(평균)이나 25%(1/4)의 값으로 Item_Weight에 존재하는 결측값
                 # 채우는 것을 추천
test.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,5656.000000,6818.000000,6818.000000,6818.000000,6818.000000
mean,12.872703,0.066121,140.419533,1997.885890,2190.941459
std,4.651034,0.051383,62.067861,8.339795,1706.131256
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.785000,0.026914,93.610050,1987.000000,836.577700
50%,12.600000,0.053799,142.448300,1999.000000,1806.648300
75%,17.000000,0.095273,185.060150,2004.000000,3115.944000
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [18]:
# 범주형 데이터 확인
# 최빈값을 확인할 때 보통 쓰임
# 이상치에 최빈값을 삽입해보는 것도 방법임

# unique : 중복값
# top : 최다 등록된 것
# freq : 빈도수

train.describe(include='O')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,6818,6818,6818,6818,4878,6818,6818
unique,1554,5,16,10,3,3,4
top,FDW26,Low Fat,Snack Foods,OUT046,Medium,Tier 3,Supermarket Type1
freq,9,4092,963,763,2228,2664,4474


In [20]:
# 결측치 확인
train.isnull().sum()
test.isnull().sum()
# 수치형 결측치 = 평균값, 중앙값 채우고 적으면 삭제
# if 이상치가 범주형이면 최빈값으로 아니면 초기값 부여

Item_Identifier                0
Item_Weight                  301
Item_Fat_Content               0
Item_Visibility                0
Item_Type                      0
Item_MRP                       0
Outlet_Identifier              0
Outlet_Establishment_Year      0
Outlet_Size                  470
Outlet_Location_Type           0
Outlet_Type                    0
dtype: int64

In [22]:
#총 결측치 확인
train.isnull().sum().sum()
test.isnull().sum().sum()

771

# 3. 데이터 전처리
결측치 처리: Item_Weight, Outlet_Size
범주형 데이터: Label Encoding 적용
불필요한 변수 제거: Item_Identifier

In [2]:
# y값 분할(타깃 분리)
# target = train.pop('Item_Outlet_Sales')

print(train.shape, test.shape)

(6818, 11) (1705, 11)


In [ ]:
# 결측값 채우기(무게 : 최소값, 사이즈 : 평균값으로 채우기)

In [3]:
train.describe()
# 무게 결측값을 최소값으로 채우기
                      #시리즈로 나온 값을 최소값으로 채움..?
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].min())
                                            #fillna(최소값): 결측값 채울 때 사용하는 함수
train.describe()
# Item_Weight: 5656 -> 6818
# 빈 값(결측값)이 최소값으로 채워짐!


# 사이즈 결측값을 최빈값으로 채우기
train.describe(include='O')
# 최빈값 : 'Outlet_Size'
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])
train.info() #전부 6818건이 됨!




#test도 똑같이!
test['Item_Weight'] = test['Item_Weight'].fillna(test['Item_Weight'].min())
test['Outlet_Size'] = test['Outlet_Size'].fillna(test['Outlet_Size'].mode()[0])
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6818 non-null   object 
 1   Item_Weight                6818 non-null   float64
 2   Item_Fat_Content           6818 non-null   object 
 3   Item_Visibility            6818 non-null   float64
 4   Item_Type                  6818 non-null   object 
 5   Item_MRP                   6818 non-null   float64
 6   Outlet_Identifier          6818 non-null   object 
 7   Outlet_Establishment_Year  6818 non-null   int64  
 8   Outlet_Size                6818 non-null   object 
 9   Outlet_Location_Type       6818 non-null   object 
 10  Outlet_Type                6818 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 586.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1705 entries, 0 to 1704
Data columns (total 11 colu

In [31]:
# 이상치 - 생략

In [32]:
train

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,NCR06,12.500,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1
1,FDW11,12.600,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1
2,FDH32,12.800,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1
3,FDL52,6.635,Regular,0.046351,Frozen Foods,37.4506,OUT017,2007,Medium,Tier 2,Supermarket Type1
4,FDO09,13.500,Regular,0.125170,Snack Foods,261.4910,OUT013,1987,High,Tier 3,Supermarket Type1
...,...,...,...,...,...,...,...,...,...,...,...
6813,NCC54,17.750,Low Fat,0.097863,Health and Hygiene,239.3196,OUT049,1999,Medium,Tier 1,Supermarket Type1
6814,NCF19,13.000,Low Fat,0.035252,Household,48.0034,OUT018,2009,Medium,Tier 3,Supermarket Type2
6815,FDU11,4.785,Low Fat,0.092576,Breads,121.0098,OUT035,2004,Small,Tier 2,Supermarket Type1
6816,FDK14,6.980,Low Fat,0.041189,Canned,82.8934,OUT045,2002,Medium,Tier 2,Supermarket Type1


In [4]:
# 레이블인코딩(원-핫 인코딩)
# 범주형 데이터가 수치형 데이터로 바뀌어져있음(ex> Low Fat : 1, Regular:2)

from sklearn.preprocessing import LabelEncoder
cols = train.columns[train.dtypes == object] # 범주형 -> 수치형
for col in cols:
    le = LabelEncoder()

    #피팅되는 경우가 안 맞는 경우 각 행마다 이동할 때 train,text행을 합치기
    le.fit(pd.concat([train[col],test[col]] ,axis=0))
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])
train

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,1467,12.500,1,0.006760,9,42.8112,1,1987,0,2,1
1,1075,12.600,1,0.048741,1,60.4194,1,1987,0,2,1
2,450,12.800,1,0.075997,6,97.1410,1,1987,0,2,1
3,613,6.635,2,0.046351,5,37.4506,2,2007,1,1,1
4,695,13.500,2,0.125170,13,261.4910,1,1987,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
6813,1292,17.750,1,0.097863,8,239.3196,9,1999,1,0,1
6814,1317,13.000,1,0.035252,9,48.0034,3,2009,1,2,2
6815,975,4.785,1,0.092576,1,121.0098,6,2004,2,1,1
6816,549,6.980,1,0.041189,3,82.8934,7,2002,1,1,1


In [5]:
# 컬럼 제거
          #'컬럼명'
train.drop('Item_Identifier', axis=1, inplace=True) # axis= 방향, 0:행, 1:열
                                                    # inplace=True(기본값 : False)
                                                    # 메서드를 실행하면 원본 데이터프레임(df) 자체가 직접 수정
test.drop('Item_Identifier', axis=1, inplace=True)
train

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,12.500,1,0.006760,9,42.8112,1,1987,0,2,1
1,12.600,1,0.048741,1,60.4194,1,1987,0,2,1
2,12.800,1,0.075997,6,97.1410,1,1987,0,2,1
3,6.635,2,0.046351,5,37.4506,2,2007,1,1,1
4,13.500,2,0.125170,13,261.4910,1,1987,0,2,1
...,...,...,...,...,...,...,...,...,...,...
6813,17.750,1,0.097863,8,239.3196,9,1999,1,0,1
6814,13.000,1,0.035252,9,48.0034,3,2009,1,2,2
6815,4.785,1,0.092576,1,121.0098,6,2004,2,1,1
6816,6.980,1,0.041189,3,82.8934,7,2002,1,1,1


# 검증데이터 분할

In [ ]:
# 8 : 2 = 학습용(train) : 검증용(val)

In [6]:
# train - val(8,2)
from sklearn.model_selection import train_test_split

                        #y_val : 예측된 결과와 비교될 예정
X_train, X_val, y_train, y_val = train_test_split(
    train, # X값
    target, # Y값
    test_size=0.2, #검증 데이터 나눠질 때 비율
    random_state=0 #난수값 고정으로!(동일한 값)
)
X_train.shape, X_val.shape, y_train.shape, y_val.shape
# 결과 : (5454, 10), (1364, 10), (5454,), (1364,)

((5454, 10), (1364, 10), (5454,), (1364,))

# 학습 및 평가(MSE, MAE, R2...)

In [10]:
# 모델 학습 : 선형회귀

from sklearn.linear_model import LinearRegression
lr = LinearRegression() # 모델 가져오기
lr.fit(X_train, y_train) #학습
y_pred = lr.predict(X_val) # 예측



In [8]:
# 평가
# 예측값 y_val과 비교!
from sklearn.metrics import mean_squared_error #MSE(평균 제곱 오차)
from sklearn.metrics import mean_absolute_error #MAE(평균 절대 오차)
from sklearn.metrics import root_mean_squared_error #RSME(제곱 평균 오차 루트)
from sklearn.metrics import r2_score #R2Score(결정계수) 

lr_mse = mean_squared_error(y_val, y_pred)
lr_mae = mean_absolute_error(y_val, y_pred)
lr_rmse = root_mean_squared_error(y_val, y_pred)
lr_r2 = r2_score(y_val, y_pred)

# 오차 비율 (10% 내외면 ㄱㅊ!)
# MSE
print("MSE : ", lr_mse) # 오차 : 선형회귀에서 정답과의 거리(오차)를 제곱해 평균을 낸 값
                        # 제곱하는 이유는 오차 중 음수가 나올 수 있기 때문에 일단 양수로 다 만들어주기 위해서
                        # 값이 클수록 예측이 실제와 다름

# MAE
print("MAE : ", lr_mae) # 오차를 절대값으로 계산(음수를 양수화시킴)
                        # 값이 클수록 예측이 실제와 다름

#RSME
print("RMSE : ", lr_rmse)

#R2Score
print("R2 : ", lr_r2)


# MSE / MAE / RSME 상대적인 기준
# 평균적으로 Y 평균값의 10% 이하 : 준수한 성능
# 평균적으로 Y 평균값의 10-30% : 보통
# Y 평균값의 30%이상 : 낮은 성능
# EX 시험점수(0-100) 예측 -> RMSE = 10(10점오차 발생) - 문제생김(10%이상)
# EX 아파트가격(30평 - 평군 5억) 예측 -> RMSE = 1000만원오차 발생 -> 오차작음(2%) : 괜찮은 모델


# R2(결정계수)
# 0.7 이상 : 좋은 모델
# 0.5 - 0.7 : 보통
# 0.3 - 0.5 : 낮은 모델

MSE :  1258220.3398377183
MAE :  858.9613052108202
RMSE :  1121.7042122759985
R2 :  0.5153323554636462


In [ ]:
# 모델학습 : LIGHTGBM사용

In [9]:
# LIGHTGBM 학습
# category로 빼서 쓰면 인코디을 할 필요가 없음

import lightgbm as lgb
model = lgb.LGBMRegressor(random_state=0, verbose=-1) #verbose=-1 : 학습 중 로그 메시지 출력 X(조용하게 실행)
model.fit(X_train, y_train)
y_pred_2 = model.predict(X_val)





MSE :  1111526.442702285
MAE :  732.8448115843846
RMSE :  1054.289544054329
R2 :  0.571838981005607


In [ ]:
# 성능지표확인
lgm_mse = mean_squared_error(y_val, y_pred_2)
lgm_mae = mean_absolute_error(y_val, y_pred_2)
lgm_rmse = root_mean_squared_error(y_val, y_pred_2)
lgm_r2 = r2_score(y_val, y_pred_2)



print("MSE : ", lgm_mse)
print("MAE : ", lgm_mae)
print("RMSE : ", lgm_rmse)
print("R2 : ", lgm_r2)

# -----------------------
# 상관계수 확인
# -----------------------

In [19]:


#제거할 컬럼


['Item_Weight',
 'Item_Visibility',
 'Item_MRP',
 'Outlet_Identifier',
 'Outlet_Establishment_Year',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [21]:
# 임계값 조절

TARGET_CORR_TH = 0.03 # 상관계수 임계값(0.03인 컬럼들만 유지할 예정)

# 숫자형 기준으로 상관계수 계산
df_corr = train.copy()
df_corr['__target__'] = y_train # y값이 없는 상태라 y값을 포함시킴

corr_with_target = df_corr.corr(numeric_only=True)['__target__'].drop('__target__')
                                #y값 기준으로 수치형 범주 확인 # 자기자신을 제외하고 나머지 
                                # x와 y와의 관계여부를 확인할 때 사용
                                # target과 나머지 간의 상관관계 확인
corr_with_target

# 유지할 컬럼(= 타깃과의 절대 상관이 임계값 이상인 컬럼만 추출)
keep_features = corr_with_target[corr_with_target.abs() >= TARGET_CORR_TH ] .index.tolist()
                                                # abs() : 음수는 절대값으로 처리
                                                # abs() >= 0.03인 컬럼의 이름을 리스트로 빼냄
# y값을 빼낼 때는 y값에 연관이 있는 컬럼들을 빼내야함
keep_features

#제거할 컬럼
problem_features = corr_with_target[corr_with_target.abs() < TARGET_CORR_TH ] .index.tolist()
problem_features

['Item_Fat_Content', 'Item_Type']

In [22]:
# ------------------------
# 컬럼 제거
# ------------------------
train.drop('Item_Fat_Content', axis=1, inplace=True)
train.drop('Item_Type', axis=1, inplace=True)

test.drop('Item_Fat_Content', axis=1, inplace=True)
test.drop('Item_Type', axis=1, inplace=True)

In [23]:
# ----------------------------------
# 확인
# ----------------------------------
train.shape, test.shape

((6818, 8), (1705, 8))

In [ ]:
# 검증 데이터 분할

In [24]:
from sklearn.model_selection import train_test_split

                        #y_val : 예측된 결과와 비교될 예정
X_train_re, X_val_re, y_train_re, y_val_re = train_test_split(
    train, # X값
    target, # Y값
    test_size=0.2, #검증 데이터 나눠질 때 비율
    random_state=0 #난수값 고정으로!(동일한 값)
)
X_train_re.shape, X_val_re.shape, y_train_re.shape, y_val_re.shape

((5454, 8), (1364, 8), (5454,), (1364,))

In [25]:
# LIGHTGBM 학습
import lightgbm as lgb
model = lgb.LGBMRegressor(random_state=0, verbose=-1) #verbose=-1 : 학습 중 로그 메시지 출력 X(조용하게 실행)
model.fit(X_train_re, y_train_re)
y_pred_2_re = model.predict(X_val_re)

In [26]:
# 성능지표확인
lgm_mse_re = mean_squared_error(y_val_re, y_pred_2_re)
lgm_mae_re = mean_absolute_error(y_val_re, y_pred_2_re)
lgm_rmse_re = root_mean_squared_error(y_val_re, y_pred_2_re)
lgm_r2_re = r2_score(y_val_re, y_pred_2_re)



print("MSE : ", lgm_mse_re)
print("MAE : ", lgm_mae_re)
print("RMSE : ", lgm_rmse_re)
print("R2 : ", lgm_r2_re)
# 값이 낮을수록 좋음!

MSE :  1108334.9408928182
MAE :  736.8110133347378
RMSE :  1052.7748766440136
R2 :  0.5730683504694064
